In [ ]:
model="provider-3/gpt-4o-mini"
api_key="ddc-a4f-350c1c62c987412cabb6fd702c17fc9b"
base_url="https://api.a4f.co/v1"

: 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
# LLM for extraction (structured output)
extraction_llm = ChatOpenAI(
    model=model,
    api_key=api_key,
    base_url=base_url,
    temperature=0.1,  # Low temp for consistent extraction
)

# LLM for synthesis (more creative)
synthesis_llm = ChatOpenAI(
    model=model,
    api_key=api_key,
    base_url=base_url,
    temperature=0.3,
)

: 

In [4]:
synthesis_llm.invoke("Hi, This is Bhanu")

AIMessage(content='Hi Bhanu! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 6, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'provider-3/gpt-4o-mini', 'system_fingerprint': None, 'id': 'ddc-a4f-chatcmpl-78b146d5b2964da89b319f73b294f2f2', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d4038d39-5827-4462-b526-4c568008055c-0', usage_metadata={'input_tokens': 6, 'output_tokens': 11, 'total_tokens': 17, 'input_token_details': {}, 'output_token_details': {}})

In [5]:
from typing import TypedDict, List, Dict, Any, Optional
# Define the state schema as per documentation
class PlanState(TypedDict, total=False):
    job: Dict[str, Any]  # job_match from API
    review: Dict[str, Any]  # resume_analysis.review
    
    # Processing state
    extracted_skills: List[str]
    present_skills: List[str]
    gaps: List[str]
    priorities: Dict[str, List[str]]
    
    # Output state
    basic_plan: List[Dict[str, Any]]
    advanced_plan: List[Dict[str, Any]]
    priority_analysis: Dict[str, Any]
    score_improvements: Dict[str, int]
    estimated_duration: str
    
    # Control flags
    rag_enabled: bool
    rationale: str
    logs: List[str]
    error: Optional[str]

# Test state initialization
initial_state = PlanState(
    job={"title": "Full Stack Engineer", "company": "Autopilot", 
         "full_description": "... expertise in React, Next.js, TypeScript, NestJS ..."},
    review={"skills_score": 8, "experience_score": 7, "clarity_score": 6, 
           "missing_skills": ["Machine Learning", "Cloud Computing"], 
           "strong_points": ["Docker", "Kubernetes"]},
    timeline_months=3,
    experience_level="intermediate",
    rag_enabled=False,
    logs=[]
)
initial_state

{'job': {'title': 'Full Stack Engineer',
  'company': 'Autopilot',
  'full_description': '... expertise in React, Next.js, TypeScript, NestJS ...'},
 'review': {'skills_score': 8,
  'experience_score': 7,
  'clarity_score': 6,
  'missing_skills': ['Machine Learning', 'Cloud Computing'],
  'strong_points': ['Docker', 'Kubernetes']},
 'timeline_months': 3,
 'experience_level': 'intermediate',
 'rag_enabled': False,
 'logs': []}

In [7]:
from langchain_core.output_parsers import JsonOutputParser
def extract_skills(state: PlanState) -> PlanState:
    """Extract skills from job description - rule-first, LLM fallback"""
    logs = state.get("logs", []) + ["extract_skills: starting"]
    
    # Rule-based extraction first (deterministic)
    jd_text = state["job"]["full_description"].lower()
    skill_patterns = {
        "React": ["react", "reactjs", "react.js"],
        "Next.js": ["next.js", "nextjs"],
        "TypeScript": ["typescript", "ts"],
        "NestJS": ["nestjs", "nest.js"],
        "Node.js": ["node.js", "nodejs"],
        "Python": ["python"],
        "Docker": ["docker"],
        "Kubernetes": ["kubernetes", "k8s"],
        "AWS": ["aws"],
        "Azure": ["azure"],
        "GCP": ["gcp"],
        "Git": ["git"],
        "GitHub": ["github"],
        "GitLab": ["gitlab"],
        "MYSQL":["mysql", "sql"],
        "PostgreSQL":["postgresql"],
        "MongoDB":["mongodb"]
        
    }
    
    extracted = []
    for skill, patterns in skill_patterns.items():
        if any(pattern in jd_text for pattern in patterns):
            extracted.append(skill)
    
    # If rule-based found enough skills, use them
    if len(extracted) >= 3:
        logs.append(f"extract_skills: rule-based found {extracted}")
        return {**state, "extracted_skills": extracted, "logs": logs}
    
    # LLM fallback for complex descriptions
    try:
        prompt = PromptTemplate.from_template("""
        Extract technical skills from this job description. Return only JSON with a "skills" array.
        Focus on programming languages, frameworks, tools, and technologies.
        
        Job Description: {job_description}
        
        Return format: {{"skills": ["skill1", "skill2"]}}
        """)
        
        chain = prompt | extraction_llm | JsonOutputParser()
        result = chain.invoke({"job_description": state["job"]["full_description"]})
        
        extracted = result.get("skills", [])[:10]  # Limit to top 10
        logs.append(f"extract_skills: LLM fallback found {extracted}")
        
    except Exception as e:
        logs.append(f"extract_skills: LLM failed, using empty list: {str(e)}")
        extracted = []
    
    return {**state, "extracted_skills": extracted, "logs": logs}

# Test the node
test_result = extract_skills(initial_state)
print("Extracted skills:", test_result["extracted_skills"])
print("Logs:", test_result["logs"][-2:])  # Show last 2 logs

Extracted skills: ['React', 'Next.js', 'TypeScript', 'NestJS']
Logs: ['extract_skills: starting', "extract_skills: rule-based found ['React', 'Next.js', 'TypeScript', 'NestJS']"]


In [ ]:
def ground_resume(state: PlanState) -> PlanState:
    """Ground resume capabilities from structured analysis - deterministic"""
    logs = state.get("logs", []) + ["ground_resume: deriving present skills"]
    
    review = state["review"]
    
    # Present skills from strong_points and role_fit_analysis
    present_skills = review.get("strong_points", [])
    
    # Add skills mentioned in role_fit_analysis if available
    role_fit = review.get("role_fit_analysis", "")
    if "frontend" in role_fit.lower() or "react" in role_fit.lower():
        present_skills.extend(["JavaScript", "HTML", "CSS"])
    if "backend" in role_fit.lower() or "node" in role_fit.lower():
        present_skills.extend(["JavaScript", "APIs"])
    
    # Remove duplicates and normalize
    present_skills = list(set(present_skills))
    
    # Experience level adjustment
    exp_level = state.get("experience_level", "intermediate")
    if exp_level == "beginner":
        # Add foundational skills
        present_skills.extend(["Programming Fundamentals", "Web Basics"])
    
    logs.append(f"ground_resume: present skills = {present_skills}")
    
    return {**state, "present_skills": present_skills, "logs": logs}

# Test the node
test_state = ground_resume(initial_state)
print("Present skills:", test_state["present_skills"])